In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import sqlite3
%matplotlib inline
from IPython.core.display import HTML
pd.options.display.max_columns = 50
import seaborn as sb
import datetime
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
HTML("<style>.container { width:100% !important; }</style>")


In [32]:
sql_conn = sqlite3.connect('flask_app/data/analysed_data.sql')
final_pivot = pd.read_sql_query('''SELECT * FROM voornamen_pivot''', con = sql_conn)
final_pivot.head()

,index,Name,Sex,Region,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,all_ages,...,Mininum_all_ages,Maximum_all_ages,Total_plus_64,Mininum_plus_64,Maximum_plus_64,Total_between18and64,Mininum_between18and64,Maximum_between18and64,Total_minus18,Mininum_minus18,Maximum_minus18,log_all_ages,Score_original,Score_vintage,Score_trend,quad_trend_a,quad_trend_b,quad_trend_c,lin_trend_a,lin_trend_b,Score_classic,last_5_years,last_5_years_min,last_5_years_max,Score_popular
0,0,Aagje,F,Belgie,0.0,10.0,7.0,6.0,5.0,5.0,12.0,10.0,8.0,10.0,19.0,11.0,10.0,11.0,12.0,11.0,8.0,11.0,7.0,0.0,325.0,...,5.0,154184.0,1098965.0,5.0,95891.0,3314571.0,5.0,56712.0,1029492.0,5.0,13873.0,5.786897,0.5,1.5,0.0,NaN,NaN,NaN,NaN,NaN,2.0,37.0,0.0,3333.0,2.5
1,1,Aagje,F,Vlaanderen,0.0,10.0,7.0,6.0,5.0,0.0,12.0,10.0,8.0,10.0,19.0,10.0,10.0,11.0,12.0,11.0,8.0,11.0,7.0,0.0,318.0,...,5.0,120289.0,659591.0,5.0,82368.0,1882638.0,5.0,37353.0,558512.0,5.0,7427.0,5.765191,0.5,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2.0,37.0,0.0,1835.0,2.5
2,2,Aaliyah,F,Belgie,0.0,0.0,0.0,0.0,0.0,0.0,11.0,18.0,22.0,19.0,14.0,22.0,17.0,20.0,24.0,18.0,18.0,20.0,26.0,31.0,227.0,...,5.0,154184.0,1098965.0,5.0,95891.0,3314571.0,5.0,56712.0,1029492.0,5.0,13873.0,5.429346,0.5,0.0,3.5,0.255661,3.606631,29.416204,1.510258,27.439790,0.0,113.0,0.0,3333.0,3.0
3,3,Aaliyah,F,Brussel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,23.0,...,5.0,11813.0,85286.0,5.0,5084.0,331078.0,5.0,7870.0,105883.0,5.0,1569.0,3.178054,2.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,10.0,0.0,557.0,2.0
4,4,Aaliyah,F,Vlaanderen,0.0,0.0,0.0,0.0,0.0,0.0,5.0,10.0,14.0,10.0,8.0,10.0,8.0,7.0,14.0,9.0,9.0,7.0,10.0,11.0,115.0,...,5.0,120289.0,659591.0,5.0,82368.0,1882638.0,5.0,37353.0,558512.0,5.0,7427.0,4.753590,1.0,0.0,1.5,0.043441,0.503004,10.326669,0.146795,9.990842,0.0,46.0,0.0,1835.0,2.5


In [311]:
feedback = pd.read_sql_query('''SELECT * FROM feedback''', con = sql_conn)
feedback.tail()

,feedback,name,session_ID,time,user_ID,sex
129,like,Anna,wfOMF0dQvVHDBJ6WVW0UEy+HBL/i+Z3Y,2016-09-03 19:07:54.064312,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
130,like,Lexie,ABrWe3hGuFXuVVtm4MpepdGtY9C86jvB,2016-09-03 19:17:29.622628,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
131,like,Lizzy,KOWFp+pnrie7uwPwPd8sFBhCfrzvOxYD,2016-09-03 19:17:52.806587,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
132,no_like,Käte,KOWFp+pnrie7uwPwPd8sFBhCfrzvOxYD,2016-09-03 19:18:00.670360,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F
133,no_like,Emily,YQX8Tns+Od8pm35OPXZk0m3NAoyDcSY8,2016-09-03 19:20:48.799656,Y3oH5KYznsVQR0MN5l9qyMcH+t5wFgLI,F


## Calculate more features

In [282]:
all_names = final_pivot[['Name', 'Score_original','Score_vintage', 'Score_classic','Score_trend', 'Score_popular', 'Sex', 'Region']]
all_names = all_names.loc[all_names['Sex'] == 'F',:]
all_names = all_names.loc[all_names['Region'] == 'Belgie',:]
all_names = all_names.drop(['Sex', 'Region'], axis = 1)
# Last 3 letters dummified
all_names = all_names.copy()
all_names['last_3_letters'] = all_names['Name'].str[-3:]
common_last_three_letters = all_names['Name'].str[-3:].value_counts()[:20].index
all_names.loc[~all_names['last_3_letters'].isin(common_last_three_letters),'last_3_letters'] = 'not_common'
all_names = pd.concat([all_names, pd.get_dummies(all_names['last_3_letters'])], axis=1)
all_names = all_names.drop(['last_3_letters', 'not_common'], axis=1)
print(all_names.shape)
# Koppel teken
all_names['dash'] = all_names['Name'].str.contains('-')
# é
all_names['é'] = all_names['Name'].str.contains('é')
# è
all_names['è'] = all_names['Name'].str.contains('è')
# ä
all_names['ä'] = all_names['Name'].str.contains('ä')
# ë
all_names['ë'] = all_names['Name'].str.contains('ë')
# ï
all_names['ï'] = all_names['Name'].str.contains('ï')
# ü
all_names['ü'] = all_names['Name'].str.contains('ï')
# Lengte
all_names['length'] = all_names['Name'].str.len()
all_names.head()


(22310, 26)


,Name,Score_original,Score_vintage,Score_classic,Score_trend,Score_popular,ana,ane,ela,ena,ida,ien,ina,ine,ita,lia,lie,lla,lle,nda,nia,nna,nne,ria,rie,tte,dash,é,è,ä,ë,ï,ü,length
0,Aagje,0.5,1.5,2.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,5
2,Aaliyah,0.5,0.0,0.0,3.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,7
6,Aaricia,1.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,7
13,Abby,1.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,4
15,Abbygail,2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,False,False,False,False,8


## Merge with feedback to create learning matrix

In [283]:
learning_matrix = pd.merge(feedback_caro, all_names, how = 'left', left_on = 'name', right_on = 'Name')
learning_matrix = learning_matrix.dropna()
learning_matrix['feedback'] = learning_matrix['feedback']=='like'
learning_matrix['feedback'] = learning_matrix['feedback'].astype(float)
learning_matrix['feedback'].value_counts()
#learning_matrix

0.0    88
1.0    18
Name: feedback, dtype: int64

In [284]:
feature_names = ['Score_original','Score_vintage','Score_classic','Score_trend','Score_popular', 'dash','é','è','ä','ë','ï','ü','length']
feature_names.extend(common_last_three_letters)
target_name = 'feedback' 

## Classificatie model

In [287]:
model_name = 'RF'

if(model_name == 'NN'):
    NN = np.arange(1,5)
    model = KNeighborsClassifier()
    clf = GridSearchCV(estimator=model, param_grid=dict(n_neighbors=NN), scoring='recall')
    clf.fit(learning_matrix[feature_names], learning_matrix[target_name])
elif(model_name =='RF'):
    n_estimators = [100]
    max_depth = np.arange(1, 10)
    model = RandomForestClassifier()
    clf = GridSearchCV(estimator=model, param_grid=dict(n_estimators=n_estimators,max_depth=max_depth), scoring='recall')
    clf.fit(learning_matrix[feature_names], learning_matrix[target_name])
    model = RandomForestClassifier(n_estimators=clf.best_estimator_.n_estimators, max_depth=clf.best_estimator_.max_depth)
    model.fit(X = learning_matrix[feature_names], y = learning_matrix[target_name])
    #print(model.feature_importances_)
    print(clf.best_score_)
    print('Max depth = %i , n estimators = %i' %(clf.best_estimator_.max_depth,clf.best_estimator_.n_estimators ))

# Tryout
test_sample = all_names.loc[all_names['Score_original'] != 4,:]
test_sample = test_sample.sample(10000).copy()
test_sample['prediction'] = clf.predict(X = test_sample[feature_names])
if(model_name=='NN'):
    print(test_sample.loc[test_sample['prediction']==1,'Name'].head(50).values)
elif(model_name=='RF'):
    test_sample['prediction_proba'] = clf.predict_proba(X = test_sample[feature_names])[:,1]
    test_sample = test_sample.sort_values(by = 'prediction_proba', axis=0, ascending=False)
    print(test_sample.head(50)['Name'].values)

0.608490566038
Max depth = 8 , n estimators = 100
['Louise' 'Capucine' 'Alexandra' 'Marion' 'Margot' 'Elise' 'Laura' 'Sarah'
 'Elisa' 'Marie' 'Juliette' 'Yasmine' 'Fatima' 'Emilie' 'Sophie' 'Elodie'
 'Faustine' 'Amélie' 'Aline' 'Roxanne' 'Jasmine' 'Melissa' 'Alyssia'
 'Marieke' 'Helena' 'Apolline' 'Lotte' 'Morgane' 'Alexia' 'Alicia' 'Laure'
 'Margaux' 'Alessia' 'Sabrina' 'Yasmina' 'Michelle' 'Belinay' 'Johanna'
 'Lily-Rose' 'Hannelore' 'Hanne' 'Marianne' 'Lucie' 'Isaline' 'Sara'
 'Babette' 'Sirine' 'Noémie' 'Bernadette' 'Emmeline']


In [271]:
pd.DataFrame({'importance':model.feature_importances_, 'feature':feature_names}).sort_values('importance', ascending=False)

,feature,importance
4,Score_popular,0.213261
2,Score_classic,0.179552
1,Score_vintage,0.166553
12,length,0.096039
3,Score_trend,0.083595
14,ine,0.060694
0,Score_original,0.050467
13,ina,0.042236
21,tte,0.017905
16,nne,0.017330


In [248]:
feature_names

['Score_original',
 'Score_vintage',
 'Score_classic',
 'Score_trend',
 'Score_popular',
 'dash',
 'é',
 'è',
 'ä',
 'ë',
 'ï',
 'ü',
 'ina',
 'ine',
 'ana',
 'nne',
 'ane',
 'lle',
 'lia',
 'lla',
 'tte',
 'nna',
 'ita',
 'ien',
 'ena',
 'nia',
 'nda',
 'ela',
 'lie',
 'ria',
 'ida',
 'rie']

## Test some new random names

,Name,Score_original,Score_vintage,Score_classic,Score_trend,Score_popular,prediction
13101,Soumaya,0.5,0.0,0.0,2.5,3.0,1.0
77108,Raimunda,2.5,0.0,0.0,0.0,0.0,0.0
29838,Colienne,1.5,0.0,0.0,0.0,0.0,0.0
49044,Ismigyul,3.5,0.0,0.0,0.0,0.0,0.0
65559,Maë,2.0,0.0,0.0,0.0,0.0,0.0
60005,Lousine,3.5,0.0,0.0,0.0,0.0,0.0
5917,Isabelle,0.5,0.5,4.0,0.0,2.5,1.0
62082,Maliha,2.0,0.0,0.0,0.0,0.0,0.0
23274,Aybuke,3.0,0.0,0.0,0.0,0.0,0.0
5406,Herlinde,0.5,1.5,2.5,0.0,0.5,0.0
